In [1]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def find_eligible_content(max_auditors=5):
    """
    Connects to Tel_QA database and finds content IDs with fewer than
    max_auditors total (real audits + pending placeholders).
    """
    client = get_mongo_client()
    db = client["Tel_QA"]
    
    qa_col     = db["QA_pairs"]
    audit_col  = db["audit_logs"]
    assign_col = db["assignment_placeholders"]
    
    # All content IDs defined in QA_pairs
    content_ids = qa_col.distinct("content_id")
    
    eligible = []
    for cid in content_ids:
        real_count    = audit_col.count_documents({"content_id": cid})
        pending_count = assign_col.count_documents({"content_id": cid})
        total         = real_count + pending_count
        
        if total < max_auditors:
            eligible.append({
                "content_id": cid,
                "real_audits": real_count,
                "pending": pending_count,
                "total": total
            })
    
    return eligible

def main():
    eligible = find_eligible_content(max_auditors=5)
    if not eligible:
        print("✅ All content audited!")
    else:
        print("Content IDs still available for auditing (fewer than 5 assignments):")
        for item in eligible:
            print(f" - ID {item['content_id']}: {item['real_audits']} audits, "
                  f"{item['pending']} pending, total {item['total']}")

if __name__ == "__main__":
    main()


✅ All content audited!


In [2]:
#!/usr/bin/env python3
import os
from pymongo import MongoClient

def get_mongo_client():
    """
    Retrieves MongoDB URI from the environment variable MONGO_URI.
    """
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise RuntimeError("Please set the MONGO_URI environment variable.")
    return MongoClient(uri)

def find_left_content(max_auditors=5):
    """
    Returns a list of dicts for each content_id that has been audited by
    fewer than `max_auditors` *distinct* interns.
    """
    client = get_mongo_client()
    db     = client["Tel_QA"]
    qa_col    = db["QA_pairs"]
    audit_col = db["audit_logs"]

    # 1) List all content IDs
    all_ids = qa_col.distinct("content_id")

    left = []
    for cid in all_ids:
        # 2) How many unique interns have audited this content?
        interns = audit_col.distinct("intern_id", {"content_id": cid})
        count   = len(interns)

        if count < max_auditors:
            left.append({
                "content_id":   cid,
                "distinct_audits": count
            })

    return left

def main():
    left = find_left_content(max_auditors=5)

    total_ids = len(left) + sum(1 for _ in find_left_content(max_auditors=0))  # hack to get total
    # Alternatively, fetch total separately:
    # total_ids = len(get_mongo_client()["Tel_QA"]["QA_pairs"].distinct("content_id"))

    if not left:
        print("✅ All content audited!")
    else:
        print(f"IDs Left: {len(left)} out of {total_ids}\n")
        for item in sorted(left, key=lambda x: x["distinct_audits"]):
            print(f" - ID {item['content_id']}: {item['distinct_audits']} distinct audits")

if __name__ == "__main__":
    main()


IDs Left: 77 out of 77

 - ID 493: 1 distinct audits
 - ID 397: 2 distinct audits
 - ID 410: 2 distinct audits
 - ID 440: 2 distinct audits
 - ID 488: 2 distinct audits
 - ID 494: 2 distinct audits
 - ID 314: 3 distinct audits
 - ID 323: 3 distinct audits
 - ID 350: 3 distinct audits
 - ID 384: 3 distinct audits
 - ID 390: 3 distinct audits
 - ID 403: 3 distinct audits
 - ID 414: 3 distinct audits
 - ID 421: 3 distinct audits
 - ID 448: 3 distinct audits
 - ID 468: 3 distinct audits
 - ID 496: 3 distinct audits
 - ID 510: 3 distinct audits
 - ID 516: 3 distinct audits
 - ID 525: 3 distinct audits
 - ID 532: 3 distinct audits
 - ID 533: 3 distinct audits
 - ID 537: 3 distinct audits
 - ID 538: 3 distinct audits
 - ID 540: 3 distinct audits
 - ID 547: 3 distinct audits
 - ID 549: 3 distinct audits
 - ID 560: 3 distinct audits
 - ID 567: 3 distinct audits
 - ID 568: 3 distinct audits
 - ID 576: 3 distinct audits
 - ID 85: 4 distinct audits
 - ID 301: 4 distinct audits
 - ID 304: 4 distinc